In [1]:
import sys
sys.path.append("../src")
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torch.nn.functional as F

import glob
import os
from datetime import datetime
import time
import math
from tqdm import tqdm

from itertools import repeat
from torch.nn.parameter import Parameter
import collections
import matplotlib
from torch_utils import *
from models import *
from visualization import *
# matplotlib.use('Agg')

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [3]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), 
                                            torchvision.transforms.Normalize(mean=(0.0,), std=(1.0,))])

mnist_dset_train = torchvision.datasets.MNIST('./data', train=True, transform=transform, target_transform=None, download=True)
train_loader = torch.utils.data.DataLoader(mnist_dset_train, batch_size=20, shuffle=True, num_workers=0)

mnist_dset_test = torchvision.datasets.MNIST('./data', train=False, transform=transform, target_transform=None, download=True)
test_loader = torch.utils.data.DataLoader(mnist_dset_test, batch_size=20, shuffle=False, num_workers=0)

In [4]:
activation_type = "softmax"
architecture = [784, 500, 500, 10]

x,y = next(iter(train_loader))
x = x.view(x.size(0),-1).to(device).T
y_one_hot = F.one_hot(y, 10).to(device).T
lambda_ = 0.9999
epsilon = 0.1#0.5
neural_lr_start = 0.1/15
neural_lr_stop = 0.0001
neural_lr_rule = "constant"
neural_lr_decay_multiplier = 0.01
neural_dynamic_iterations = 10

lr_start = {'ff' : 0.01, "fb": 0.001}
# lr_start = {'ff' : 0.001, "fb": 0.001}

model = CorInfoMaxV3(architecture, lambda_, epsilon, activation_type)

In [5]:
trn_acc_list = []
tst_acc_list = []

n_epochs = 10
lr = lr_start
for epoch_ in range(n_epochs):
    lr = {'ff' : lr_start['ff'] * (0.9)**epoch_, 'fb' : lr_start['fb'] * (0.9)**epoch_}
    for idx, (x, y) in tqdm(enumerate(train_loader)):
        x, y = x.to(device), y.to(device)
        #x = activation_inverse(x.view(x.size(0),-1).T, activation_type)
        x = x.view(x.size(0),-1).T
        y_one_hot = F.one_hot(y, 10).to(device).T
        y_one_hot = 0.94 * y_one_hot + 0.03 * torch.ones(*y_one_hot.shape, device = device)
        _ = model.batch_step( x, y_one_hot, lr, neural_lr_start, neural_lr_stop, neural_lr_rule,
                              neural_lr_decay_multiplier, neural_dynamic_iterations, optimizer = "sgd"
                            )

    trn_acc = evaluatePC(model, train_loader, device, apply_activation_inverse = False, activation_type = activation_type, printing = False)
    tst_acc = evaluatePC(model, test_loader, device, apply_activation_inverse = False, activation_type = activation_type, printing = False)
    trn_acc_list.append(trn_acc)
    tst_acc_list.append(tst_acc)
    
    print("Epoch : {}, Train Accuracy : {}, Test Accuracy : {}".format(epoch_+1, trn_acc, tst_acc))

3000it [11:46,  4.25it/s]
0it [00:00, ?it/s]

Epoch : 1, Train Accuracy : 0.11236666666666667, Test Accuracy : 0.1135


3000it [14:51,  3.36it/s]
0it [00:00, ?it/s]

Epoch : 2, Train Accuracy : 0.10218333333333333, Test Accuracy : 0.101


3000it [13:25,  3.73it/s]
1it [00:00,  6.60it/s]

...

Epoch : 8, Train Accuracy : 0.11236666666666667, Test Accuracy : 0.1135


3000it [07:21,  6.79it/s]
1it [00:00,  6.82it/s]

Epoch : 9, Train Accuracy : 0.11236666666666667, Test Accuracy : 0.1135


3000it [07:24,  6.75it/s]


Epoch : 10, Train Accuracy : 0.11236666666666667, Test Accuracy : 0.1135


In [6]:
model.Wff

array([{'weight': tensor([[1., 0., 0.,  ..., 0., 0., 0.],
               [0., 1., 0.,  ..., 0., 0., 0.],
               [0., 0., 1.,  ..., 0., 0., 0.],
               ...,
               [0., 0., 0.,  ..., 0., 0., 0.],
               [0., 0., 0.,  ..., 0., 0., 0.],
               [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0'), 'bias': tensor([[ 1.0243e-05],
               [-1.2048e-05],
               [-3.0160e-05],
               [ 2.1231e-05],
               [ 6.3887e-07],
               [ 5.4209e-05],
               [ 1.1858e-05],
               [ 3.5092e-05],
               [-4.8703e-05],
               [ 3.7372e-05],
               [-1.4405e-05],
               [ 3.8394e-05],
               [ 5.4900e-05],
               [ 4.5596e-05],
               [-4.3029e-06],
               [ 1.0860e-05],
               [ 1.7017e-05],
               [-1.4696e-08],
               [-2.4757e-05],
               [-2.9119e-05],
               [ 1.5763e-05],
               [ 1.6506e-05],
       

In [7]:
neurons = model.fast_forward(x)

In [8]:
layers = [x] + neurons

In [9]:
self = model

In [10]:
Wff = self.Wff
Wfb = self.Wfb
B = self.B
Bsigma = self.Bsigma
gam_ = self.gam_
one_over_epsilon = self.one_over_epsilon
        
layers = [x] + neurons  # concatenate the input to other layers
init_grads = [torch.zeros(*neurons_.shape, dtype = torch.float, device = self.device) for neurons_ in neurons]
# layers_after_activation = [list(self.activation_func(layers[jj], self.activation_type)) for jj in range(len(layers))]
layers_after_activation = [[x, 1]] + [list(self.activation_func(layers[jj], self.activation_type)) for jj in range(1, len(layers) - 1)] + [neurons[-1]]
## Compute forward errors
forward_errors = [layers[jj + 1] - (Wff[jj]['weight'] @ layers_after_activation[jj][0] + Wff[jj]['bias']) for jj in range(len(Wff))]
## Compute backward errors
backward_errors = [layers_after_activation[jj][0] - (Wfb[jj]['weight'] @ layers[jj + 1] + Wfb[jj]['bias']) for jj in range(1, len(Wfb))]


In [11]:
jj = 0
# (gam_ * Bsigma[jj]['weight'] @ (layers_after_activation[jj + 1][0])).shape#* layers_after_activation[jj + 1][1])
(backward_errors[jj] * layers_after_activation[jj+1][1])

RuntimeError: The size of tensor a (20) must match the size of tensor b (500) at non-singleton dimension 2

In [ ]:
backward_errors[jj].shape

In [ ]:
layers_after_activation[jj+1][1].shape

In [ ]:
torch.bmm(layers_after_activation[jj+1][1], backward_errors[jj].T.unsqueeze(2)).squeeze(2).T.shape

In [ ]:
layers_after_activation[jj + 1][1].shape

In [ ]:
torch.bmm(layers_after_activation[jj + 1][1], (gam_ * Bsigma[jj]['weight'] @ (layers_after_activation[jj + 1][0])).T.unsqueeze(2)).squeeze(2).T.shape

In [ ]:
torch.norm(F.softmax(x, 0)[:,0] - F.softmax(x[:,0], 0))

In [ ]:
sigmaX = F.softmax(x[:,0], 0)
J1 = (torch.diag(sigmaX) - torch.outer(sigmaX, sigmaX))
J1.shape

In [ ]:
sigmaX = F.softmax(x, 0)
J2 = torch.diag_embed(sigmaX.T) - torch.einsum('ij, ik -> ijk', sigmaX.T, sigmaX.T)
J2.shape

In [ ]:
torch.norm(J1 - J2[0])

In [ ]:
sigmaX.T.unsqueeze(2).shape

In [ ]:
J1.shape

In [ ]:
(J1 @ F.softmax(x[:,0], 0).T).shape

In [ ]:
torch.norm(torch.bmm(J2, sigmaX.T.unsqueeze(2))[0, :, 0] - J1 @ F.softmax(x[:,0], 0).T)

In [ ]:
torch.bmm(J2, sigmaX.T.unsqueeze(2)).shape

In [ ]:
(torch.diag_embed(x.T) - torch.einsum('ij, ik -> ijk', x.T, x.T)).shape

In [ ]:
trn_acc_list = []
tst_acc_list = []

n_epochs = 10
lr = lr_start
for epoch_ in range(n_epochs):
    lr = {'ff' : lr_start['ff'] * (0.9)**epoch_, 'fb' : lr_start['fb'] * (0.9)**epoch_}
    for idx, (x, y) in tqdm(enumerate(train_loader)):
        x, y = x.to(device), y.to(device)
        #x = activation_inverse(x.view(x.size(0),-1).T, activation_type)
        x = x.view(x.size(0),-1).T
        y_one_hot = F.one_hot(y, 10).to(device).T
        y_one_hot = 0.94 * y_one_hot + 0.03 * torch.ones(*y_one_hot.shape, device = device)
        _ = model.batch_step( x, y_one_hot, lr, neural_lr_start, neural_lr_stop, neural_lr_rule,
                              neural_lr_decay_multiplier, neural_dynamic_iterations, optimizer = "sgd"
                            )

    trn_acc = evaluatePC(model, train_loader, device, apply_activation_inverse = False, activation_type = activation_type, printing = False)
    tst_acc = evaluatePC(model, test_loader, device, apply_activation_inverse = False, activation_type = activation_type, printing = False)
    trn_acc_list.append(trn_acc)
    tst_acc_list.append(tst_acc)
    
    print("Epoch : {}, Train Accuracy : {}, Test Accuracy : {}".format(epoch_+1, trn_acc, tst_acc))

In [ ]:
plot_convergence_plot(trn_acc_list, xlabel = 'Number of Epochs', ylabel = 'Accuracy %',
                      title = 'PC Train Accuracy w.r.t. Epochs', 
                      figsize = (12,8), fontsize = 25, linewidth = 3)

In [ ]:
plot_convergence_plot(tst_acc_list, xlabel = 'Number of Epochs', ylabel = 'Accuracy %',
                      title = 'PC Test Accuracy w.r.t. Epochs', 
                      figsize = (12,8), fontsize = 25, linewidth = 3)

In [ ]:
plt.figure(figsize = (7,7))
plt.imshow(torch2numpy(model.B[0]["weight"]))

In [ ]:
plt.figure(figsize = (7,7))
plt.imshow(torch2numpy(model.Bsigma[1]["weight"]))

In [ ]:
model.Bsigma[1]["weight"]